In [1]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

In [2]:
location= (
            pd.read_excel(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind Dale v2.xlsx',sheet_name='location', delimiter = ';')
            .fillna('')
            .assign( Location_name=lambda x: (""+x['Location1']+" "+x['Location2']+"").str.replace(" ","_"),
                     Main_location_name=lambda x: (""+x['Location1']+"").str.replace(" ","_"),
                     Location_nr=lambda x: x.groupby('Main_location_name').ngroup(ascending=True)+1
                   )
            .sort_values(by='Main_location_name')
            .assign(Location1_nr = lambda x: x.groupby('Main_location_name').cumcount()+1,
                    n=1,
                    Row = lambda x: x['n'].cumsum()
                   )
)

location.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\location.json',orient='records')



person = (
            pd.read_excel(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind Dale v2.xlsx',sheet_name='person', delimiter = ';')
              .assign(Location2	 = lambda x: x['Location2'].fillna(''),
                      Main_location_name=lambda x: (""+x['Location1']+" "+x['Location2']+"").str.replace(" ","_"))
              .sort_values(by = ['Main_location_name','First_Name'])
              .assign(Location_nr=lambda x: x.groupby('Main_location_name').ngroup(ascending=True)+1,
                      Location1_nr=lambda x: x.groupby('Location1').cumcount()+1,
                      Location2_nr=lambda x: x.groupby('Main_location_name').cumcount()+1,
                      Row= lambda x:  np.arange(len(x)),
                      Surname= lambda x: x['Surname'].fillna(''),
                      Party_relationship= lambda x: x['Party_relationship'].fillna(''),
                      ID=lambda x: "ID"+x['First_Name']+"",
                      full_name=lambda x: x['First_Name']+"_"+x['Surname']+"")
              .sort_values(by = 'Column')
              .assign(First_faction=lambda x: (x.groupby('Column').cumcount()+1 ).apply(lambda x: 1 if x==1 else 0),
                      Faction_id=lambda x: (""+x['Faction_name']+"").str.replace(" ","_"),
                      First_alignment=lambda x:(x.groupby('Column').cumcount()+1).apply(lambda x: 1 if x==1 else 0)
                     )
              .sort_values(by = ['Main_location_name','First_Name'])       
               
    
                     )

quest= (
        pd.read_excel(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind Dale v2.xlsx',sheet_name='quest', delimiter = ';')
          .fillna('')
          .assign(Main_location_name=lambda x: (""+x['Location1']+"").str.replace(" ","_"),
                  Location_name=lambda x: (""+x['Location1']+" "+x['Location2']+"").str.replace(" ","_"),
                    n=1,
                    Row = lambda x: x['n'].cumsum()
                  )
        )
quest.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\quest.json',orient='records')  



quest_giver= ( quest
             .query("Completed==0" )
             .assign(test = lambda x: x['Quest_giver'].str.len())
             .query("test!=0")

             .rename(columns = {'Quest_giver': 'quest_giver_name'}, inplace = False)
             .assign(quest_giver=1,
                     quest_giver_underline=lambda x: (""+x['quest_giver_name']+"").str.replace(" ","_"))  
             .loc[:,['quest_giver_name','quest_giver','quest_giver_underline']]
#            .loc[:,['quest_giver_name','quest_giver','quest_giver_underline']]
  
)

quest_target= ( quest
            .query("Completed==0" )
             .assign(test = lambda x: x['Quest_target'].str.len())
             .query("test!=0")
             .rename(columns = {'Quest_target': 'quest_target_name'}, inplace = False)
             .assign(quest_target=1,
                     quest_target_underline=lambda x: (""+x['quest_target_name']+"").str.replace(" ","_"))        
             .loc[:,['quest_target_name','quest_target','quest_target_underline']]
  
)


# person.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\person.json',ori ent='records')
person = (
             pd.merge(person, quest_target,how='left', left_on='full_name', right_on='quest_target_underline')
            .drop(['quest_target_name','quest_target_underline'], axis=1)    
 )

person = (
             pd.merge(person, quest_giver,how='left', left_on='full_name', right_on='quest_giver_underline')
            .drop(['quest_giver_name','quest_giver_underline'], axis=1)
            .assign(quest_target = lambda x: x['quest_target'].fillna('0'),
                    quest_giver = lambda x: x['quest_giver'].fillna('0'),
                   )
            .sort_values(by = ['Main_location_name','First_Name'])
 )

person.to_json(r'C:\Users\joaki\Google Drive\dnd\Icewind Dale\Icewind-Dale\person.json',orient='records')  


In [4]:
quest

,Quest,Quest_giver,Quest_target,Location1,Location2,Description,Reward,Completed,Main_location_name,Location_name,n,Row
0,Take out Sephek Kaltro,Clyn,Sephek,Bremen,,Bring in Sephek Kaltro for murders. Tarrgas is...,100gp,0,Bremen,Bremen_,1,1
1,Retrive sleigh,Hruna,,Wilderness,Close to Dwarven Valley,Bring sledge back. Yeti,50 gp gems per adventure member,0,Wilderness,Wilderness_Close_to_Dwarven_Valley,1,2
2,Murdered Town Speaker,,,Good Mead,,Town Speaker has been killed by 9 feet tall gi...,,0,Good_Mead,Good_Mead_,1,3
3,Genasi adventurers,,,Caer-Konig or Cairn-Denival,,Adventuring bands of genasi seen in eastern te...,,0,Caer-Konig_or_Cairn-Denival,Caer-Konig_or_Cairn-Denival_,1,4
4,Gareth the adventurer is missing.,Keegan,Gareth,Kelvins Cairn,,Keegans husband Gareth is missing in the slope...,,0,Kelvins_Cairn,Kelvins_Cairn_,1,5
5,Find Hoarvar and return him,Kora,Hoarvar,Caer-Konig or Cairn-Denival,Castle,Hoarvar was lost in storm. Returned with frost...,,0,Caer-Konig_or_Cairn-Denival,Caer-Konig_or_Cairn-Denival_Castle,1,6
6,Find the creature in Maer Dualdon,Tali,Plesiosaur,Maer Dualdon,,Find out about the creature in the Maer. Turns...,,,Maer_Dualdon,Maer_Dualdon_,1,7
7,Find Ravisin,,Ravisin,unkknown,,Ravisin serves the frost Maiden. We can find him.,,,unkknown,unkknown_,1,8


In [99]:
quest

,Quest,Quest_giver,Quest_target,Location1,Location2,Description,Reward,Completed,Main_location_name,Location_name,Row
0,Take out Sephek Kaltro,Clyn,Sephek Kaltro,,,Bring in Sephek Kaltro for murders,100gp,0,,_,1
1,Retrive sleigh,Hruna,,Wilderness,Close to Dwarven Valley,Bring sledge back. Yeti,50 gp gems per adventure member,0,Wilderness,Wilderness_Close_to_Dwarven_Valley,2
2,Murdered Town Speaker,,,Good Mead,,Town Speaker has been killed by 9 feet tall gi...,,0,Good_Mead,Good_Mead_,3
